In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import time

# Objectives
**out_data = FC_NN x in_data + Bias + noises**  
where, out_data and in_data are 512 x 1 matrix  
       FC_NN is 512 x 512 matrix         
       Bias is 512 x 1 matrix
       
Assume that we are not sure about FC_NN and Bias matrixes but we have 10,000 dataset of in_data and out_data. 

**Let auto discover these FC_NN and Bias matrixes using Adam Optimizer from TensorFlow**

In [2]:
in_data = []
out_data = []
dataset_size = 10000
FC_NN = np.random.rand(512,512)
Bias_NN = np.random.rand(512,1)-0.5
tmp = 0
for i in range(dataset_size):
    in_data.append(np.random.rand(512,1))
for i in range(dataset_size):        
    out_data.append(((np.dot(FC_NN,in_data[i])/512)- np.ones((512,1))*0.25)*100 - Bias_NN - (np.random.rand(512,1)-0.5)*0.001)
np_in_data = np.array(in_data)
np_out_data = np.array(out_data)

**Let look at np_in and np_out**

In [3]:
print('np_in_data[0,0:10] : {}'.format(np_in_data[0,0:10].reshape((1,10))))
print('np_out_data[0,0:10] : {}'.format(np_out_data[0,0:10].reshape((1,10))))

np_in_data[0,0:10] : [[0.36749441 0.62200434 0.57885494 0.81593413 0.27238199 0.0821933
  0.93716219 0.1708414  0.59749138 0.16194285]]
np_out_data[0,0:10] : [[ 2.07452833  0.51479488  1.50930054  0.72834451  1.61812541  0.59299283
  -0.24792461  2.32092347  1.88108148 -0.30848034]]


In [4]:
def randomize(x, y):
    """ Randomizes the order of data samples and their corresponding labels"""
    permutation = np.random.permutation(y.shape[0])
    shuffled_x = x[permutation]
    shuffled_y = y[permutation]
    return shuffled_x, shuffled_y

def train_test_split(x,y,train_ratio):
    train_size = int(y.shape[0]*train_ratio)
    x_train = x[0:train_size]
    y_train = y[0:train_size]
    x_test = x[train_size:]
    y_test = y[train_size:]
    return x_train, y_train, x_test, y_test


# Setting up the model in TensorFlow

In [5]:
learning_rate = 0.001
str_save_name = 'model_4.ckpt'

# Create placeholder
mt = tf.Variable(tf.random_normal([512, 512], stddev=0.35),
                      name="weights")
Bias = tf.Variable(tf.random_normal([512, 1], stddev=0.35),
                      name="Bias")

# Build the model
x = tf.placeholder(tf.float32, shape=[512, 1])
y = tf.placeholder(tf.float32, shape=[512, 1])
pred = ((tf.matmul(mt, x)/512) - np.ones((512,1))*0.25)*100 - Bias

# Train the model
loss = tf.reduce_sum(tf.pow(pred-y, 2))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

# # Experiment Dropout implementation (development in progress)
# dropout = tf.placeholder(tf.float32, shape=[512, 512])
# pred_train = ((tf.matmul(tf.multiply(mt,dropout), x)/512) - np.ones((512,1))*0.25)*100 - Bias
# loss_train = tf.reduce_sum(tf.pow(pred_train-y, 2))
# optimizer_dropout = tf.train.AdamOptimizer(learning_rate).minimize(loss_train)

Instructions for updating:
Colocations handled automatically by placer.


**Save the model and weight to model_2.ckpt**

In [6]:
saver = tf.train.Saver()
init_op = tf.initialize_all_variables()
with tf.Session() as sess:    
    sess.run(init_op)     
    save_path = saver.save(sess, str_save_name)
    print("initial model file created!")

Instructions for updating:
Use `tf.global_variables_initializer` instead.
initial model file created!


# Train the model
**Reload the model and train the model. The model will be saved in every 10 Epochs. In this way, we can stop training at any time and can continue from the last saved one.**

In [7]:
# parameters
train_ratio = 0.8
training_epochs = 100
dropout_rate = 0
display_step = 2
save_step = 8

# Train and Test Split
np_in_data_train, np_out_data_train, np_in_data_test, np_out_data_test = train_test_split(np_in_data, 
                                                                                          np_out_data, 
                                                                                          train_ratio = train_ratio)

# Start training
with tf.Session() as sess:    
    
    # Load the model
    saver.restore(sess, str_save_name)    

    for epoch in range(training_epochs):               
        start = time.time()        
        # randomize train dataset
        np_in_data_train, np_out_data_train = randomize(np_in_data_train, np_out_data_train)
        
        # Train the model for one epoch
        for n in range(np_out_data_train.shape[0]):      
            if(n%100==0):
                dropout_1 = (np.random.rand(512,512)>dropout_rate)*1.0        
            sess.run(optimizer,feed_dict={x:np_in_data_train[n],y:np_out_data_train[n]})

        # Display logs per epoch step
        if (epoch) % display_step == 0:
            
            # get total cost of train set
            c = 0;
            for n in range(np_out_data_train.shape[0]):
                c = c + sess.run(loss,feed_dict={x:np_in_data_train[n],y:np_out_data_train[n]})
            c = c/np_out_data_train.shape[0]
            
            # get total cost of test set
            c_t = 0;
            for n in range(np_out_data_test.shape[0]):
                c_t = c_t + sess.run(loss,feed_dict={x:np_in_data_test[n],y:np_out_data_test[n]})
            c_t = c_t/np_out_data_test.shape[0]
            
            # Show Absolute  Error
            predict = sess.run(pred,feed_dict={x:np_in_data_test[0]})
            
            # Display the message
            print("epoch:", '%04d' % (epoch+1), '/%d' % (training_epochs), ", avg train cost=", "{:.6}".format(str(c)),
                  ", avg test cost=", "{:.6}".format(str(c_t)), 
                  ", abs_err [idx 0]:", "{:.6}".format(str(sum(sum(abs(predict-np_out_data_test[0]))))),
                  ", cyc time:", "{:.6}".format(str(time.time() - start)), "s")

        if (epoch) % save_step == 0:            
            
            # Save the model
            save_path = saver.save(sess, str_save_name)
            print("Model Saved!")
            
    print("Optimization Finished!")
    predict = sess.run(pred,feed_dict={x:np_in_data_test[0]})
    save_path = saver.save(sess, str_save_name)
    print("Model Saved!")
    print("abs_err of idx 0: {}".format(sum(abs(predict-np_out_data_test[0]))))

    

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from model_4.ckpt
epoch: 0001 /100 , avg train cost= 130.52 , avg test cost= 132.75 , abs_err [idx 0]: 212.81 , cyc time: 16.440 s
Model Saved!
epoch: 0003 /100 , avg train cost= 7.6751 , avg test cost= 9.0784 , abs_err [idx 0]: 55.804 , cyc time: 16.265 s
epoch: 0005 /100 , avg train cost= 0.6358 , avg test cost= 0.6914 , abs_err [idx 0]: 18.755 , cyc time: 16.224 s
epoch: 0007 /100 , avg train cost= 0.1790 , avg test cost= 0.1818 , abs_err [idx 0]: 2.5645 , cyc time: 16.254 s
epoch: 0009 /100 , avg train cost= 0.1402 , avg test cost= 0.1415 , abs_err [idx 0]: 10.497 , cyc time: 16.315 s
Model Saved!
epoch: 0011 /100 , avg train cost= 0.1423 , avg test cost= 0.1472 , abs_err [idx 0]: 7.1702 , cyc time: 16.223 s
epoch: 0013 /100 , avg train cost= 0.2039 , avg test cost= 0.2009 , abs_err [idx 0]: 5.8217 , cyc time: 16.310 s
epoch: 0015 /100 , avg train cost= 0.1065

# Check the result
**Print Predicted FC_NN matrix vs actual one**  
**Print Predicted Bias matrix vs actual one**  
**You will find that they are actually very close**  

In [8]:
with tf.Session() as sess: 
    saver.restore(sess, str_save_name)
    print(sess.run(mt)[0:2][0:2])
    print(FC_NN[0:2][0:2])
    print(sess.run(Bias)[0:10].reshape((1,10)))
    print(Bias_NN[0:10].reshape((1,10)))
    print('predict[0,0:10] : {}'.format(predict[0:10].reshape((1,10))))
    print('np_out_data[0,0:10] : {}'.format(np_out_data[0,0:10].reshape((1,10))))

INFO:tensorflow:Restoring parameters from model_4.ckpt
[[0.15019812 0.19793849 0.27795264 ... 0.82029885 0.21533759 0.97129107]
 [0.12732624 0.4783094  0.30039266 ... 0.85725963 1.0064675  0.7586497 ]]
[[0.16197962 0.20607511 0.28786249 ... 0.82202146 0.2227817  0.97544313]
 [0.12576    0.48723765 0.30444728 ... 0.85682352 0.99933494 0.76461612]]
[[-0.4426669  -0.12216051 -0.18660145 -0.02668742 -0.47945103 -0.0317819
   0.1468918  -0.17940368  0.32288018 -0.05600573]]
[[-0.43955023 -0.11924664 -0.18566434 -0.03373253 -0.47388131 -0.0315662
   0.1487314  -0.17631927  0.32587368 -0.05849385]]
predict[0,0:10] : [[ 1.4177333  -0.71064603  0.32624322 -0.3030112   0.81652427 -0.48463422
  -1.6597307   1.1757281   0.9540123  -1.2611377 ]]
np_out_data[0,0:10] : [[ 2.07452833  0.51479488  1.50930054  0.72834451  1.61812541  0.59299283
  -0.24792461  2.32092347  1.88108148 -0.30848034]]
